In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from PIL import Image


In [ ]:
def load_images_from_directory(data_path, label):
    images = []
    labels = []
    
    for filename in os.listdir(data_path):
        img_path = os.path.join(data_path, filename)
        img = Image.open(img_path).convert('L') # Convert to grayscale
        img_array = np.array(img) / 255.0 # Normalize pixel values
        images.append(img_array)
        labels.append(label)
    
    return images, labels


In [ ]:
train_path = "/content/drive/MyDrive/GurNum/train"
val_path = "/content/drive/MyDrive/GurNum/val"

X_train, y_train = [], []
X_val, y_val = [], []

for i in range(10):
    train_data_path = os.path.join(train_path, str(i))
    val_data_path = os.path.join(val_path, str(i))
    
    train_images, train_labels = load_images_from_directory(train_data_path, i)
    val_images, val_labels = load_images_from_directory(val_data_path, i)
    
    X_train.extend(train_images)
    y_train.extend(train_labels)
    X_val.extend(val_images)
    y_val.extend(val_labels)

X_train = np.array(X_train).reshape(-1, 32*32)
X_val = np.array(X_val).reshape(-1, 32*32)
y_train = np.array(y_train)
y_val = np.array(y_val)


In [ ]:
print(f"X TRAIN: {X_train.shape}")
print(f"Y TRAIN: {y_train.shape}")
print(f"X VAL: {X_val.shape}")
print(f"Y TRAIN: {y_val.shape}")

X TRAIN: (1000, 1024)
Y TRAIN: (1000,)
X VAL: (178, 1024)
Y TRAIN: (178,)


In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)


In [ ]:
print(f"X VAL: {X_val.shape}")
print(f"Y TRAIN: {y_val.shape}")

X VAL: (178, 1024)
Y TRAIN: (178, 10)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_model():
    model = Sequential()
    model.add(Dense(128, input_dim=1024, activation='relu'))  # Input layer
    model.add(Dense(64, activation='relu'))  # Hidden layer
    model.add(Dense(10, activation='softmax'))  # Output layer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()


In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
32/32 [==============================] - 1s 11ms/step - loss: 1.1967 - accuracy: 0.6450 - val_loss: 0.6442 - val_accuracy: 0.8539
Epoch 2/20
32/32 [==============================] - 0s 5ms/step - loss: 0.2956 - accuracy: 0.9170 - val_loss: 0.3796 - val_accuracy: 0.8876
Epoch 3/20
32/32 [==============================] - 0s 6ms/step - loss: 0.1773 - accuracy: 0.9480 - val_loss: 0.3179 - val_accuracy: 0.9157
Epoch 4/20
32/32 [==============================] - 0s 5ms/step - loss: 0.1123 - accuracy: 0.9710 - val_loss: 0.2978 - val_accuracy: 0.9157
Epoch 5/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0795 - accuracy: 0.9840 - val_loss: 0.2619 - val_accuracy: 0.9326
Epoch 6/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0501 - accuracy: 0.9920 - val_loss: 0.2605 - val_accuracy: 0.9382
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0450 - accuracy: 0.9920 - val_loss: 0.2096 - val_accuracy: 0.9382
Epoch 8/20
32/32 [=

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")


6/6 [==============================] - 0s 3ms/step - loss: 0.2311 - accuracy: 0.9494
Validation Loss: 0.23105914890766144
Validation Accuracy: 0.949438214302063


### FROM THE SCRATCH

In [ ]:
import numpy as np


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=1, keepdims=True)


In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = sigmoid(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = softmax(self.Z2)
        return self.A2

    def backward(self, X, y, learning_rate):
        m = X.shape[0]
        dZ2 = self.A2 - y
        dW2 = 1 / m * np.dot(self.A1.T, dZ2)
        db2 = 1 / m * np.sum(dZ2, axis=0, keepdims=True)
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * sigmoid_derivative(self.A1)
        dW1 = 1 / m * np.dot(X.T, dZ1)
        db1 = 1 / m * np.sum(dZ1, axis=0, keepdims=True)

        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            self.backward(X, y, learning_rate)


In [ ]:
input_size = 1024
hidden_size = 64
output_size = 10

nn = NeuralNetwork(input_size, hidden_size, output_size)
nn.train(X_train, y_train, epochs=1000, learning_rate=0.01)


In [ ]:
def evaluate(model, X, y):
    y_pred = model.forward(X)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y, axis=1)
    accuracy = np.mean(y_pred_labels == y_true_labels)
    return accuracy

In [ ]:
accuracy = evaluate(nn, X_val, y_val)
print(f"Validation Accuracy: {accuracy}")


Validation Accuracy: 0.34831460674157305
